In [1]:
import csv
from osgeo import gdal
import fiona
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely import wkt
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, CSV
import folium

### Wildfire query

In [34]:
sparql = SPARQLWrapper("https://stko-kwg.geog.ucsb.edu/graphdb/repositories/KWG")
sparql.setReturnFormat(CSV)

fire_query = """PREFIX gnis: <http://gnis-ld.org/lod/gnis/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX usgs: <http://gnis-ld.org/lod/usgs/ontology/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX time: <http://www.w3.org/2006/time#>
select * where { 
    ?fire rdf:type kwg-ont:Fire .
    ?fire kwg-ont:sfWithin kwgr:Earth.North_America.United_States.USA.5.42_1 .
    ?fire kwg-ont:hasTemporalScope ?temporal_scope .
    ?temporal_scope time:inXSDgYear ?year .
    ?fire kwg-ont:hasIncidentName ?name .
}"""
sparql.setQuery(fire_query)
try:
    res = sparql.query().convert()
    with open('fires.csv','wb') as file:
        file.write(res)
except Exception as e:
    print(f"There was a problem retrieving the fires: {e}")

There was a problem retrieving the fires: HTTP Error 502: Bad Gateway


### Road query

In [5]:
road_query = """PREFIX sf: <http://www.opengis.net/ont/sf#>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
select ?sign ?wkt where { 
    ?s rdf:type <http://stko-kwg.geog.ucsb.edu/lod/ontology/RoadSegment> .
    ?s rdfs:label ?road .
    ?s kwg-ont:hasRoadSign ?sign .
    ?s kwg-ont:sfWithin kwgr:Earth.North_America.United_States.USA.5.42_1 .
    ?s geo:hasGeometry ?geometry .
    ?geometry geosparql:asWKT ?wkt .
}"""
sparql.setQuery(road_query)
try:
    res = sparql.query().convert()
    with open('roads.csv','wb') as file:
        file.write(res)
except Exception as e:
    print(f"There was a problem retrieving the roads: {e}")

There was a problem retrieving the roads: HTTP Error 502: Bad Gateway


### Hospital query

In [12]:
hospital_query = """PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX geosparql: <http://www.opengis.net/ont/geosparql#>
select ?hospital ?has_helipad ?name ?bed_count ?wkt where { 
    ?hospital a kwg-ont:Hospital .
    ?hospital kwg-ont:helipadAvailability ?has_helipad .
    ?hospital kwg-ont:hospitalName ?name .
    ?hospital kwg-ont:noHospitalBeds ?bed_count .
    ?hospital geosparql:hasDefaultGeometry ?geom .
    ?geom geosparql:asWKT ?wkt .
    kwgr:Earth.North_America.United_States.USA.5.42_1 kwg-ont:sfContains ?s2Cell.
    ?hospital kwg-ont:sfWithin ?s2Cell .
}"""
sparql.setQuery(hospital_query)
try:
    res = sparql.query().convert()
    with open('hospitals.csv','wb') as file:
            file.write(res)
except Exception as e:
    print(f"There was a problem retrieving the hospitals: {e}")

There was a problem retrieving the hospitals: HTTP Error 502: Bad Gateway


### Park query

In [13]:
park_query = """PREFIX gnis: <http://gnis-ld.org/lod/gnis/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX usgs: <http://gnis-ld.org/lod/usgs/ontology/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
select * where { 
    ?park rdf:type usgs:Park .
    ?park kwg-ont:sfWithin kwgr:Earth.North_America.United_States.USA.5.42_1 .
    ?park gnis:officialName ?name
}"""
sparql.setQuery(park_query)
try:
    res = sparql.query().convert()
    with open('parks.csv','wb') as file:
            file.write(res)
except Exception as e:
    print(f"There was a problem retrieving the parks: {e}")

There was a problem retrieving the parks: HTTP Error 502: Bad Gateway


### School query

In [14]:
school_query = """PREFIX gnis: <http://gnis-ld.org/lod/gnis/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX usgs: <http://gnis-ld.org/lod/usgs/ontology/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
select * where { 
    ?school rdf:type usgs:School .
    ?school kwg-ont:sfWithin kwgr:Earth.North_America.United_States.USA.5.42_1 .
    ?school gnis:officialName ?name
}"""
sparql.setQuery(school_query)
try:
    res = sparql.query().convert()
    with open('schools.csv','wb') as file:
            file.write(res)
except Exception as e:
    print(f"There was a problem retrieving the schools: {e}")

There was a problem retrieving the schools: HTTP Error 502: Bad Gateway


### Load shapefile

In [2]:
santa_barbara_shape = gpd.read_file('shapefile/SB_County.shp')
#santa_barbara_shape.crs

In [3]:
init_x = (min(santa_barbara_shape.bounds['minx']) + max(santa_barbara_shape.bounds['maxx']))/2
init_y = (min(santa_barbara_shape.bounds['miny']) + max(santa_barbara_shape.bounds['maxy']))/2

sb_map = folium.Map(location=[init_y, init_x], zoom_start=8)
folium.GeoJson(data=santa_barbara_shape, name='Santa Barbara County').add_to(sb_map)
sb_map

In [4]:
def convert_gdf(df):
    df['wkt'] = df['wkt'].apply(wkt.loads)
    output_gdf = gpd.GeoDataFrame(geometry=df['wkt'], crs='epsg:4326')
    output_gdf = output_gdf.to_crs(santa_barbara_shape.crs)
    return output_gdf

In [5]:
line_strings = pd.read_csv('roads.csv')
hospital_locations = pd.read_csv('hospitals.csv')
line_data = convert_gdf(line_strings)
hospital_data = convert_gdf(hospital_locations)

In [6]:
# add roads
feature_group1 = folium.FeatureGroup("Roads")
for _, r in line_data.iterrows():
    line_string = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    road = line_string.to_json()
    road = folium.GeoJson(data=road,style_function=lambda x: {'color': 'red','weight': 2})
    feature_group1.add_child(road)

# add hospitals
feature_group2 = folium.FeatureGroup("Hospitals")
for idx, row in hospital_locations.iterrows():
    iframe = folium.IFrame(str(row['name'])+'<br>'+'Bed Count: '+str(row['bed_count']))
    popup = folium.Popup(iframe, min_width=200, max_width=300)
    feature_group2.add_child(folium.Marker(location=[row['wkt'].y, row['wkt'].x], 
                                          popup=popup)) 

# add schools

# add parks

# add wildfire polygons

sb_map.add_child(feature_group1)
sb_map.add_child(feature_group2)

# try different base map layers
folium.TileLayer('Cartodb Positron').add_to(sb_map)
folium.TileLayer('Stamen Toner').add_to(sb_map)

# add layer control
folium.LayerControl().add_to(sb_map)
sb_map